## CESM2 - LARGE ENSEMBLE (LENS2)

- Notebook to compare AMOC from LENS2 with other datasets

### Imports

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np 
import cf_xarray
from matplotlib.ticker import MaxNLocator
import cftime
import nc_time_axis
import matplotlib.pyplot as plt
import warnings, getpass, os
import distributed
import ncar_jobqueue
import scipy.io
%matplotlib inline
from matplotlib.offsetbox import AnchoredText

### Dask

In [ ]:
cluster = ncar_jobqueue.NCARCluster()
cluster.scale(20)
client = distributed.Client(cluster)
client

## Part1
- In this first part, we will make the comparison of the maximum current function of the AMOC from LENS2 with different reanalyses.

### Data

In [ ]:
# Reanalysis (Copernicus) and Large Ensemble (CESM2)
t1='1993-01-01' # Inital time
t2='2020-12-31' # Final time
models=('glor','foam','cglo','oras','all_models','lens2') # Abbreviations for the models
for im in range(0,len(models)):
    if im<=4:
        st=f'ds_{models[im]}_amoc = xr.open_dataset(\'/glade/scratch/mauricio/Data/AMOC/glorys2v2/{models[im]}_amoc.nc\')'; exec(st); del(st)
        st=f'ds_{models[im]}_amoc.coords[\'depth\']=ds_{models[im]}_amoc.coords[\'depth\']*0.001'; exec(st); del(st)
        st=f'ds_{models[im]}_amoc=ds_{models[im]}_amoc.amoc.sel(time=slice(t1,t2))'#.resample(time=\'1Y\', closed=\'left\').mean(\'time\')' 
        exec(st); del(st)  
    else:
        st=f'ds_{models[im]}_amoc = xr.open_dataset(\'/glade/scratch/mauricio/Data/AMOC/LENS2/{models[im]}_amoc.nc\')'; exec(st); del(st)
        ds_lens2_amoc.coords['moc_z']=ds_lens2_amoc.coords['moc_z']*0.00001 # +cm to km
        ds_lens2_amoc=ds_lens2_amoc.rename({'moc_z': 'depth', 'lat_aux_grid': 'latitude'})
        st=f'ds_{models[im]}_amoc=ds_{models[im]}_amoc.amoc.sel(time=slice(t1,t2))'#.resample(time=\'1Y\', closed=\'left\').mean(\'time\')' 
        exec(st); del(st)
    #    ds_lens2_amoc.coords['time']=ds_glor_amoc.coords['time']

In [ ]:
# Altimeter (Streamfunction).
t1='1993-01-31T00:00:00.000000000' # Inital time
t2='2020-12-31T00:00:00.000000000' # Final time
ds = scipy.io.loadmat('/glade/scratch/mauricio/Data/AMOC/Dong_et_al_JGR_2021/strfun_all.mat')
timexr = pd.date_range("1993-01-15", periods=348, freq='M')
ds_alt_strfun = xr.merge([xr.Dataset({'strfun': 
                        xr.DataArray(data=ds.get('strfun_26N'),dims=['depth','time'],
                                                             coords={'depth':np.transpose(ds.get('zz_26N')).squeeze()*-1,'time': timexr})}),
                        xr.Dataset({'strfun20': xr.DataArray(data=ds.get('strfun_20S'),dims=['depth','time'],
                                                            coords={'depth':np.transpose(ds.get('zz')).squeeze()*-1,'time': timexr})}),
                        xr.Dataset({'strfun25': xr.DataArray(data=ds.get('strfun_25S'),dims=['depth','time'],
                                                              coords={'depth':np.transpose(ds.get('zz')).squeeze()*-1,'time': timexr})}),
                        xr.Dataset({'strfun30': xr.DataArray(data=ds.get('strfun_30S'),dims=['depth','time'],
                                                              coords={'depth':np.transpose(ds.get('zz')).squeeze()*-1,'time': timexr})}),
                        xr.Dataset({'strfun35': xr.DataArray(data=ds.get('strfun_35S'),dims=['depth','time'],
                                                              coords={'depth':np.transpose(ds.get('zz')).squeeze()*-1,'time': timexr})})]) 
teste=xr.concat([ds_alt_strfun.strfun,ds_alt_strfun.strfun20,ds_alt_strfun.strfun25,ds_alt_strfun.strfun30,ds_alt_strfun.strfun35], 'latitude')
ds_alt_strfun=xr.concat([ds_alt_strfun.strfun,ds_alt_strfun.strfun20,ds_alt_strfun.strfun25,ds_alt_strfun.strfun30,ds_alt_strfun.strfun35], pd.Index(
    [26, -20,-25, -30, -35], name='latitude')); del teste
ds_alt_strfun=ds_alt_strfun/1000000 # Get Sverdrup as a unit
ds_alt_strfun.coords['depth']=ds_alt_strfun.coords['depth']*-0.001 # to get positive depths and to transfor m to km
ds_alt_strfun=ds_alt_strfun.sel(time=slice(t1,t2))#.resample(time='1Y', closed='left').mean('time')

In [ ]:
def calculate_ticks(ax, ticks, round_to=0.01, center=False):
    upperbound = np.ceil(ax.get_ybound()[1]/round_to)
    lowerbound = np.floor(ax.get_ybound()[0]/round_to)
    dy = upperbound - lowerbound
    fit = np.floor(dy/(ticks - 1)) + 1
    dy_new = (ticks - 1)*fit
    if center:
        offset = np.floor((dy_new - dy)/2)
        lowerbound = lowerbound - offset
    values = np.linspace(lowerbound, lowerbound + dy_new, ticks)
    return values*round_to

### Plot 2

In [ ]:
# Parameters
lw=1
plt.rcParams.update({'font.size': 20})
t1='1993-01-01' # initial time
t2='2020-12-31' # final time
lat=[26.5,-20,-25,-30,-34.5]
a=[0,0,1,1,2]; b=[0,1,0,1,0]

# Data
models=('glor','foam','cglo','oras','all_models') #,'alt','lens2')
leg=('GLORYS','FOAM','CGLO','ORAS','Models_mean') #,'Altimeter','LENS2')
colors=('darkgreen','darkred','orange','orchid','turquoise')  #,'blue','black')

# Plot
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(15,10),sharey=False)
for il in range(0,len(lat)):
    st=f'ds_lens2_amoc.sel(latitude={lat[il]},method=\'nearest\').squeeze().sel(time=slice(t1,t2)).mean(dim=\'time\').plot.line(ax=axs[{a[il]},{b[il]}],y=\'depth\',alpha=0.01,color=\'k\',linewidth=lw,label=None,add_legend=False)'; exec(st); del st # all members
    for im in range(0,len(models)):
        st=f'ds_{models[im]}_amoc.sel(latitude={lat[il]},method=\'nearest\').sel(time=slice(t1,t2)).mean(dim=\'time\').plot.line(ax=axs[{a[il]},{b[il]}],y=\'depth\',linewidth=lw,color=\'{colors[im]}\',label=\'{leg[im]}\')'; exec(st); del(st) # Reanalysis
    st=f'ds_alt_strfun.sel(latitude={lat[il]},method=\'nearest\').squeeze().sel(time=slice(t1,t2)).mean(dim=\'time\').plot.line(ax=axs[{a[il]},{b[il]}],y=\'depth\',color=\'darkblue\',linewidth=lw,label=\'Altimeter\')'; exec(st); del st # all members
    st=f'ds_lens2_amoc.sel(latitude={lat[il]},method=\'nearest\').squeeze().mean(dim=\'member_id\').sel(time=slice(t1,t2)).mean(\'time\').plot.line(ax=axs[{a[il]},{b[il]}],y=\'depth\',color=\'k\',linewidth=lw,label=\'LENS2\')'; exec(st); del st # all members
    plt.ylabel('Depth [km]')
    st=f'axs[{a[il]},{b[il]}].set_yticks(np.arange(0,6,step=1))'; exec(st); del st
    st=f'axs[{a[il]},{b[il]}].set_xticks(np.arange(-8,26,step=4))'; exec(st); del st
    st=f'axs[{a[il]},{b[il]}].set_xlim([-8,26])'; exec(st); del st
    st=f'axs[{a[il]},{b[il]}].set_ylim([0,6])'; exec(st); del st
    st=f'axs[{a[il]},{b[il]}].set_ylabel(\'Depth [km]\',fontsize=24)'; exec(st); del st
    st=f'axs[{a[il]},{b[il]}].set_xlabel(\'AMOC Streamfunction [Sv]\',fontsize=24)'; exec(st); del st
    st=f'axs[{a[il]},{b[il]}].grid()'; exec(st); del st
    #st=f'axs[{a[il]},{b[il]}].set_title(\'Latitude: {lat[il]}\',fontsize=14)'; exec(st); del st
    st=f'axs[{a[il]},{b[il]}].set_title(None)'; exec(st); del st

# Config plots    
plt.tight_layout()
axs[2,0].legend(loc='center left', bbox_to_anchor=(1.4, 0.2), title = 'Datasets:', fontsize=17,title_fontsize=20)
axs[2][1].set_visible(False)
axs[0][0].set_xticklabels([]); axs[0][1].set_xticklabels([]); axs[1][0].set_xticklabels([])
axs[0][1].set_yticklabels([]); axs[1][1].set_yticklabels([]); 
axs[0][1].set(ylabel=None); axs[1][1].set(ylabel=None); 
axs[0][0].set(xlabel=None); axs[0][1].set(xlabel=None); axs[1][0].set(xlabel=None)
plt.subplots_adjust(wspace=0.02, hspace=0.03)
#plt.invert_yaxis()
axs[0][0].invert_yaxis(); axs[0][1].invert_yaxis()
axs[1][0].invert_yaxis(); axs[1][1].invert_yaxis()
axs[2][0].invert_yaxis()

# Identification per letters
at = AnchoredText("A", prop=dict(size=18), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
axs[0][0].add_artist(at)

at = AnchoredText("B", prop=dict(size=18), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
axs[0][1].add_artist(at)

at = AnchoredText("C", prop=dict(size=18), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
axs[1][0].add_artist(at)

at = AnchoredText("D", prop=dict(size=18), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
axs[1][1].add_artist(at)

at = AnchoredText("E", prop=dict(size=18), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
axs[2][0].add_artist(at)

# Save Plot
plt.savefig('vertical_profile.png',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
# Reanalysis (Copernicus) and Large Ensemble (CESM2)
t1='1990-01-01' # Inital time
t2='2020-12-31' # Final time
ds_lens2_amoc_past = xr.open_dataset('/glade/scratch/mauricio/Data/AMOC/LENS2/lens2_amoc.nc')
ds_lens2_amoc_past.coords['moc_z']=ds_lens2_amoc_past.coords['moc_z']*0.00001 # +cm to km
ds_lens2_amoc_past=ds_lens2_amoc_past.rename({'moc_z': 'depth', 'lat_aux_grid': 'latitude'})
ds_lens2_amoc_past=ds_lens2_amoc_past.amoc.sel(time=slice(t1,t2)) 

t1='2070-01-01' # Inital time
t2='2100-12-31' # Final time
ds_lens2_amoc_future = xr.open_dataset('/glade/scratch/mauricio/Data/AMOC/LENS2/lens2_amoc.nc')
ds_lens2_amoc_future.coords['moc_z']=ds_lens2_amoc_future.coords['moc_z']*0.00001 # +cm to km
ds_lens2_amoc_future=ds_lens2_amoc_future.rename({'moc_z': 'depth', 'lat_aux_grid': 'latitude'})
ds_lens2_amoc_future=ds_lens2_amoc_future.amoc.sel(time=slice(t1,t2)) 

In [ ]:
# Parameters
lw=2
plt.rcParams.update({'font.size': 20})
lat=[26.5,-20,-25,-30,-34.5]
a=[0,0,1,1,2]; b=[0,1,0,1,0]

# Plot
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(15,10),sharey=False)
for il in range(0,len(lat)):
    #st=f'ds_lens2_amoc_past.sel(latitude={lat[il]},method=\'nearest\').squeeze().sel(time=slice(t1,t2)).mean(dim=\'time\').plot.line(ax=axs[{a[il]},{b[il]}],y=\'depth\',alpha=0.01,color=\'k\',linewidth=lw,label=None,add_legend=False)'; exec(st); del st # all members
    st=f'ds_lens2_amoc_past.sel(latitude={lat[il]},method=\'nearest\').squeeze().mean(dim=\'member_id\').mean(\'time\').plot.line(ax=axs[{a[il]},{b[il]}],y=\'depth\',color=\'blue\',linewidth=lw,label=\'1990-2020\')'; exec(st); del st # all members
    #st=f'ds_lens2_amoc_future.sel(latitude={lat[il]},method=\'nearest\').squeeze().sel(time=slice(t1,t2)).mean(dim=\'time\').plot.line(ax=axs[{a[il]},{b[il]}],y=\'depth\',alpha=0.01,color=\'darkgreen\',linewidth=lw,label=None,add_legend=False)'; exec(st); del st # all members
    st=f'ds_lens2_amoc_future.sel(latitude={lat[il]},method=\'nearest\').squeeze().mean(dim=\'member_id\').mean(\'time\').plot.line(ax=axs[{a[il]},{b[il]}],y=\'depth\',color=\'red\',linewidth=lw,label=\'2070-2100\')'; exec(st); del st # all members    
    plt.ylabel('Depth [km]')
    st=f'axs[{a[il]},{b[il]}].set_yticks(np.arange(0,6,step=1))'; exec(st); del st
    st=f'axs[{a[il]},{b[il]}].set_xticks(np.arange(-5,20,step=2.5))'; exec(st); del st
    st=f'axs[{a[il]},{b[il]}].set_xlim([-5,20])'; exec(st); del st
    st=f'axs[{a[il]},{b[il]}].set_ylim([0,6])'; exec(st); del st
    st=f'axs[{a[il]},{b[il]}].set_ylabel(\'Depth [km]\',fontsize=24)'; exec(st); del st
    st=f'axs[{a[il]},{b[il]}].set_xlabel(\'AMOC Streamfunction [Sv]\',fontsize=24)'; exec(st); del st
    st=f'axs[{a[il]},{b[il]}].grid()'; exec(st); del st
    #st=f'axs[{a[il]},{b[il]}].set_title(\'Latitude: {lat[il]}\',fontsize=14)'; exec(st); del st
    st=f'axs[{a[il]},{b[il]}].set_title(None)'; exec(st); del st

# Config plots    
plt.tight_layout()
axs[2,0].legend(loc='center left', bbox_to_anchor=(1.4, 0.2), title = 'LENS2:', fontsize=17,title_fontsize=20)
axs[2][1].set_visible(False)
axs[0][0].set_xticklabels([]); axs[0][1].set_xticklabels([]); axs[1][0].set_xticklabels([])
axs[0][1].set_yticklabels([]); axs[1][1].set_yticklabels([]); 
axs[0][1].set(ylabel=None); axs[1][1].set(ylabel=None); 
axs[0][0].set(xlabel=None); axs[0][1].set(xlabel=None); axs[1][0].set(xlabel=None)
plt.subplots_adjust(wspace=0.05, hspace=0.03)
#plt.invert_yaxis()
axs[0][0].invert_yaxis(); axs[0][1].invert_yaxis()
axs[1][0].invert_yaxis(); axs[1][1].invert_yaxis()
axs[2][0].invert_yaxis()

# Identification per letters
at = AnchoredText("A", prop=dict(size=18), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
axs[0][0].add_artist(at)

at = AnchoredText("B", prop=dict(size=18), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
axs[0][1].add_artist(at)

at = AnchoredText("C", prop=dict(size=18), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
axs[1][0].add_artist(at)

at = AnchoredText("D", prop=dict(size=18), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
axs[1][1].add_artist(at)

at = AnchoredText("E", prop=dict(size=18), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
axs[2][0].add_artist(at)

# Save Plot
plt.savefig('vertical_profile.png',dpi=300,bbox_inches='tight')
plt.show()

#### Build a table to plot the maximum of the streamfunction, the depth of that maximum and the depth of inversion 

In [ ]:
df

In [ ]:
# Maximum
max_lens2=ds_lens2_amoc.mean('member_id').sel(latitude=26.5,method='nearest').groupby('time.year').max(dim=['time','depth'])
max_std_lens2=max_lens2.std()
max_mean_lens2=max_lens2.mean() 

In [ ]:
# Depth of the maximum 
teste=ds_lens2_amoc.mean('member_id').sel(latitude=26.5,method='nearest').groupby('time.year').max(dim=['time'])
#depth_std_lens2=max_lens2.std()
#depth_mean_lens2=max_lens2.mean() 

### Plot 1